# Applio NoUI
Created by [Blaise](https://github.com/blaise-tk) with [Vidal](https://github.com/Vidalnt) and [Poopmaster](https://github.com/poiqazwsx). Based on [RVC_CLI](https://github.com/blaise-tk/RVC_CLI).

- Colab inspired on [RVC v2 Disconnected](https://colab.research.google.com/drive/1XIPCP9ken63S7M6b5ui1b36Cs17sP-NS).



### If you restart the runtime, run it again.

In [ ]:
%cd /content/Applio

# Installation
## If the runtime restarts, run the cell above and re-run the installation steps.

In [ ]:
# @title Mount Google Drive
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# @title Clone
!git clone https://github.com/IAHispano/Applio
%cd /content/Applio

In [ ]:
# @title Install
rot_47 = lambda encoded_text: "".join(
    [
        (
            chr(
                (ord(c) - (ord("a") if c.islower() else ord("A")) - 47) % 26
                + (ord("a") if c.islower() else ord("A"))
            )
            if c.isalpha()
            else c
        )
        for c in encoded_text
    ]
)
import codecs
import os
import shutil
import tarfile
import subprocess
from pathlib import Path
from datetime import datetime
E = Exception
B = print


def vidal_setup(ForceIn):
    L = "Kikpm.ovm.bu"
    K = "/content/"
    C = ForceIn

    def F():
        print("Installing pip packages...")
        subprocess.check_call(["pip", "install", "-r", "requirements.txt", "--quiet"])

    A = K + rot_47(L)
    G = K + rot_47(L)
    D = "/"
    if not os.path.exists(A):
        M = os.path.dirname(A)
        os.makedirs(M, exist_ok=True)
        print("No cached install found..")
        try:
            N = rot_47(
                codecs.decode(
                    "pbbxa://pcooqvonikm.kw/QIPqaxivw/Ixxtqw/zmawtdm/uiqv/Kwtij/Xvxcz.biz.oh",
                    "rot_13",
                )
            )
            subprocess.run(["wget", "-O", A, N])
            print("Download completed successfully!")
        except E as H:
            print(str(H))
            if os.path.exists(A):
                os.remove(A)
    if Path(A).exists():
        with tarfile.open(G, "r:gz") as I:
            for J in I.getmembers():
                O = os.path.join(D, J.name)
                try:
                    I.extract(J, D)
                except E as H:
                    print("Failed to extract a file")
                    C = True
            print(f"Extraction of {G} to {D} completed.")
        if os.path.exists(A):
            os.remove(A)
        if C:
            F()
            C = False
    else:
        F()


vidal_setup(False)
print("Finished installing requirements!")

In [ ]:
# @title Download models
!python core.py prerequisites

# Infer


In [ ]:
# @title Download model
# @markdown Hugging Face or Google Drive
model_link = "https://huggingface.co/Darwin/Darwin/resolve/main/Darwin.zip"  # @param {type:"string"}

!python core.py download --model_link "{model_link}"

In [ ]:
# @title Run Inference
# @markdown Please upload the audio file to your Google Drive path `/content/drive/MyDrive` and specify its name here. For the model name, use the zip file name without the extension. Alternatively, you can check the path `/content/Applio/logs` for the model name (name of the folder).

import os

current_dir = os.getcwd()

model_name = "Darwin"  # @param {type:"string"}
model_folder = os.path.join(current_dir, f"logs/{model_name}")

if not os.path.exists(model_folder):
    raise FileNotFoundError(f"Model directory not found: {model_folder}")

files_in_folder = os.listdir(model_folder)
pth_path = next((f for f in files_in_folder if f.endswith(".pth")), None)
index_file = next((f for f in files_in_folder if f.endswith(".index")), None)

if pth_path is None or index_file is None:
    raise FileNotFoundError("No model found.")

pth_file = os.path.join(model_folder, pth_path)
index_file = os.path.join(model_folder, index_file)

input_path = "/content/example.wav"  # @param {type:"string"}
output_path = "/content/output.wav"
export_format = "WAV"  # @param ['WAV', 'MP3', 'FLAC', 'OGG', 'M4A'] {allow-input: false}
f0method = "rmvpe"  # @param ["pm", "dio", "crepe", "crepe-tiny", "harvest", "rmvpe", "fcpe", "hybrid[rmvpe+fcpe]"] {allow-input: false}
f0up_key = 0  # @param {type:"slider", min:-24, max:24, step:0}
filter_radius = 3  # @param {type:"slider", min:0, max:10, step:0}
rms_mix_rate = 0.8  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
protect = 0.5  # @param {type:"slider", min:0.0, max:0.5, step:0.1}
index_rate = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
hop_length = 128  # @param {type:"slider", min:1, max:512, step:0}
clean_strength = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
split_audio = False  # @param{type:"boolean"}
clean_audio = False  # @param{type:"boolean"}
autotune = False  # @param{type:"boolean"}

!python core.py infer --f0up_key "{f0up_key}" --filter_radius "{filter_radius}" --index_rate "{index_rate}" --hop_length "{hop_length}" --rms_mix_rate "{rms_mix_rate}" --protect "{protect}" --f0autotune "{autotune}" --f0method "{f0method}" --input_path "{input_path}" --output_path "{output_path}" --pth_path "{pth_file}" --index_path "{index_file}" --split_audio "{split_audio}" --clean_audio "{clean_audio}" --clean_strength "{clean_strength}" --export_format "{export_format}"

from IPython.display import Audio, display, clear_output

output_path = output_path.replace(".wav", f".{export_format.lower()}")
# clear_output()
display(Audio(output_path, autoplay=True))

# Train

In [ ]:
# @title Preprocess Dataset
model_name = "Darwin"  # @param {type:"string"}
dataset_path = "/content/drive/MyDrive/Darwin_Dataset"  # @param {type:"string"}

sample_rate = "40k"  # @param ["32k", "40k", "48k"] {allow-input: false}
sr = int(sample_rate.rstrip("k")) * 1000

!python core.py preprocess --model_name "{model_name}" --dataset_path "{dataset_path}" --sampling_rate "{sr}"

In [ ]:
# @title Extract Features
rvc_version = "v2"  # @param ["v2", "v1"] {allow-input: false}
f0method = "rmvpe"  # @param ["pm", "dio", "crepe", "crepe-tiny", "harvest", "rmvpe"] {allow-input: false}
hop_length = 128  # @param {type:"slider", min:1, max:512, step:0}

sr = int(sample_rate.rstrip("k")) * 1000

!python core.py extract --model_name "{model_name}" --rvc_version "{rvc_version}" --f0method "{f0method}" --hop_length "{hop_length}" --sampling_rate "{sr}"

In [ ]:
# @title Train
# @markdown ### ➡️ Model Information
import requests
import threading
import time
import os
import shutil
import hashlib
import time

LOGS_FOLDER = "/content/Applio/logs/"
WEIGHTS_FOLDER = LOGS_FOLDER + model_name
GOOGLE_DRIVE_PATH = "/content/drive/MyDrive/RVC_Backup"


def import_google_drive_backup():
    print("Importing Google Drive backup...")
    weights_exist = False
    for root, dirs, files in os.walk(GOOGLE_DRIVE_PATH):
        for filename in files:
            filepath = os.path.join(root, filename)
            if os.path.isfile(filepath) and not filepath.startswith(
                os.path.join(GOOGLE_DRIVE_PATH, "weights")
            ):
                backup_filepath = os.path.join(
                    LOGS_FOLDER, os.path.relpath(filepath, GOOGLE_DRIVE_PATH)
                )
                backup_folderpath = os.path.dirname(backup_filepath)
                if not os.path.exists(backup_folderpath):
                    os.makedirs(backup_folderpath)
                    print(f"Created backup folder: {backup_folderpath}", flush=True)
                shutil.copy2(filepath, backup_filepath)  # copy file with metadata
                print(f"Imported file from Google Drive backup: {filename}")
            elif filepath.startswith(
                os.path.join(GOOGLE_DRIVE_PATH, "weights")
            ) and filename.endswith(".pth"):
                weights_exist = True
                weights_filepath = os.path.join(
                    WEIGHTS_FOLDER,
                    os.path.relpath(
                        filepath, os.path.join(GOOGLE_DRIVE_PATH, "weights")
                    ),
                )
                weights_folderpath = os.path.dirname(weights_filepath)
                if not os.path.exists(weights_folderpath):
                    os.makedirs(weights_folderpath)
                    print(f"Created weights folder: {weights_folderpath}", flush=True)
                shutil.copy2(filepath, weights_filepath)  # copy file with metadata
                print(f"Imported file from weights: {filename}")
    if weights_exist:
        print("Copied weights from Google Drive backup to local weights folder.")
    else:
        print("No weights found in Google Drive backup.")
    print("Google Drive backup import completed.")


def get_md5_hash(file_path):
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()


def copy_weights_folder_to_drive():
    destination_folder = os.path.join(GOOGLE_DRIVE_PATH, "weights")
    try:
        if not os.path.exists(destination_folder):
            os.makedirs(destination_folder)

        num_copied = 0
        for filename in os.listdir(WEIGHTS_FOLDER):
            if filename.endswith(".pth"):
                source_file = os.path.join(WEIGHTS_FOLDER, filename)
                destination_file = os.path.join(destination_folder, filename)
                if not os.path.exists(destination_file):
                    shutil.copy2(source_file, destination_file)
                    num_copied += 1
                    print(f"Copied {filename} to Google Drive!")

        if num_copied == 0:
            print("No new finished models found for copying.")
        else:
            print(f"Finished copying {num_copied} files to Google Drive!")

    except Exception as e:
        print(f"An error occurred while copying weights: {str(e)}")


if "autobackups" not in globals():
    autobackups = False


def backup_files():
    print("\nStarting backup loop...")
    last_backup_timestamps_path = os.path.join(
        LOGS_FOLDER, "last_backup_timestamps.txt"
    )
    fully_updated = False

    while True:
        try:
            updated = False
            last_backup_timestamps = {}

            try:
                with open(last_backup_timestamps_path, "r") as f:
                    last_backup_timestamps = dict(line.strip().split(":") for line in f)
            except FileNotFoundError:
                pass

            for root, dirs, files in os.walk(LOGS_FOLDER):
                # Excluding "zips" directory
                if "zips" in dirs:
                    dirs.remove("zips")
                if "mute" in dirs:
                    dirs.remove("mute")
                for filename in files:
                    if filename != "last_backup_timestamps.txt":
                        filepath = os.path.join(root, filename)
                        if os.path.isfile(filepath):
                            backup_filepath = os.path.join(
                                GOOGLE_DRIVE_PATH,
                                os.path.relpath(filepath, LOGS_FOLDER),
                            )
                            backup_folderpath = os.path.dirname(backup_filepath)
                            if not os.path.exists(backup_folderpath):
                                os.makedirs(backup_folderpath)
                                print(
                                    f"Created backup folder: {backup_folderpath}",
                                    flush=True,
                                )
                            last_backup_timestamp = last_backup_timestamps.get(filepath)
                            current_timestamp = os.path.getmtime(filepath)
                            if (
                                last_backup_timestamp is None
                                or float(last_backup_timestamp) < current_timestamp
                            ):
                                shutil.copy2(filepath, backup_filepath)
                                last_backup_timestamps[filepath] = str(
                                    current_timestamp
                                )
                                if last_backup_timestamp is None:
                                    print(f"Backed up file: {filename}")
                                else:
                                    print(f"Updating backed up file: {filename}")
                                updated = True
                                fully_updated = False

            for filepath in list(last_backup_timestamps.keys()):
                if not os.path.exists(filepath):
                    backup_filepath = os.path.join(
                        GOOGLE_DRIVE_PATH, os.path.relpath(filepath, LOGS_FOLDER)
                    )
                    if os.path.exists(backup_filepath):
                        os.remove(backup_filepath)
                        print(f"Deleted file: {filepath}")
                    del last_backup_timestamps[filepath]
                    updated = True
                    fully_updated = False

            if not updated and not fully_updated:
                print("Files are up to date.")
                fully_updated = True
                sleep_time = 15
            else:
                sleep_time = 0.1

            with open(last_backup_timestamps_path, "w") as f:
                for filepath, timestamp in last_backup_timestamps.items():
                    f.write(f"{filepath}:{timestamp}\n")

            time.sleep(sleep_time)

        except Exception as e:
            print(f"An error occurred: {str(e)}")


if autobackups:
    autobackups = False
    print("Autobackup Disabled")
else:
    autobackups = True
    print("Autobackup Enabled")

total_epoch = 800  # @param {type:"integer"}
batch_size = 15  # @param {type:"slider", min:1, max:25, step:0}
gpu = 0
sr = int(sample_rate.rstrip("k")) * 1000
pitch_guidance = True  # @param{type:"boolean"}
auto_backups = True  # @param{type:"boolean"}
pretrained = True  # @param{type:"boolean"}
tensorboard = True  # @param{type:"boolean"}
# @markdown ### ➡️ Choose how many epochs your model will be stored
save_every_epoch = 10  # @param {type:"slider", min:1, max:100, step:0}
save_only_latest = False  # @param{type:"boolean"}
save_every_weights = False  # @param{type:"boolean"}
overtraining_detector = False  # @param{type:"boolean"}
overtraining_threshold = 50  # @param {type:"slider", min:1, max:100, step:0}
# @markdown ### ❓ Optional
# @markdown In case you select custom pretrained, you will have to download the pretraineds and enter the path of the pretraineds.
custom_pretrained = False  # @param{type:"boolean"}
g_pretrained_path = "/content/Applio/rvc/pretraineds/pretraineds_custom/G48k.pth"  # @param {type:"string"}
d_pretrained_path = "/content/Applio/rvc/pretraineds/pretraineds_custom/D48k.pth"  # @param {type:"string"}

if "pretrained" not in globals():
    pretrained = True

if "custom_pretrained" not in globals():
    custom_pretrained = False

if "g_pretrained_path" not in globals():
    g_pretrained_path = "Custom Path"

if "d_pretrained_path" not in globals():
    d_pretrained_path = "Custom Path"


def start_train():
    if tensorboard == True:
        %load_ext tensorboard
        %tensorboard --logdir /content/Applio/logs/
    !python core.py train --model_name "{model_name}" --rvc_version "{rvc_version}" --save_every_epoch "{save_every_epoch}" --save_only_latest "{save_only_latest}" --save_every_weights "{save_every_weights}" --total_epoch "{total_epoch}" --sampling_rate "{sr}" --batch_size "{batch_size}" --gpu "{gpu}" --pitch_guidance "{pitch_guidance}" --pretrained "{pretrained}" --custom_pretrained "{custom_pretrained}" --g_pretrained_path "{g_pretrained_path}" --d_pretrained_path "{d_pretrained_path}" --overtraining_detector "{overtraining_detector}" --overtraining_threshold "{overtraining_threshold}"


server_thread = threading.Thread(target=start_train)
server_thread.start()

if auto_backups:
    backup_files()
else:
    while True:
        time.sleep(10)

In [ ]:
# @title Generate index file
!python core.py index --model_name "{model_name}" --rvc_version "{rvc_version}"

In [ ]:
# @title Save model
# @markdown Enter the name of the model and the steps. You can find it in your `/content/Applio/logs` folder.
%cd /content
import shutil, os

model_name = "Darwin"  # @param {type:"string"}
model_epoch = 800  # @param {type:"integer"}
save_big_file = False  # @param {type:"boolean"}

if os.path.exists("/content/zips"):
    shutil.rmtree("/content/zips")
print("Removed zips.")
!mkdir -p /content/zips/{model_name}/
print("Created zips.")
if f"{model_name}.pth" not in os.listdir(f"/content/Applio/weights"):
    print("There is no weight file with that name")
if not save_big_file:
    !cp /content/Applio/logs/{model_name}/added_*.index /content/zips/{model_name}/
    !cp /content/Applio/logs/{model_name}/total_*.npy /content/zips/{model_name}/
    !cp /content/Applio/weights/{model_name}.pth /content/zips/{model_name}/{model_name}{model_epoch}.pth
    %cd /content/zips
    !zip -r {model_name}.zip {model_name}
if save_big_file:
    %cd /content/Applio
    latest_steps = -1
    logs_folder = "./logs/" + model_name
    for filename in os.listdir(logs_folder):
        if filename.startswith("G_") and filename.endswith(".pth"):
            steps = int(filename.split("_")[1].split(".")[0])
            if steps > latest_steps:
                latest_steps = steps
    MODELZIP = model_name + ".zip"
    !mkdir -p /content/zips
    ZIPFILEPATH = os.path.join("/content/zips", MODELZIP)
    for filename in os.listdir(logs_folder):
        if "G_" in filename or "D_" in filename:
            if str(latest_steps) in filename:
                !zip -r {ZIPFILEPATH} {os.path.join(logs_folder, filename)}
        else:
            !zip -r {ZIPFILEPATH} {os.path.join(logs_folder, filename)}
    for filename in os.listdir("./weights"):
        if MODELNAME in filename:
            !zip -r {ZIPFILEPATH} {os.path.join('./weights/', filename)}

!mkdir -p /content/drive/MyDrive/RVC_Backup/
shutil.move(
    f"/content/zips/{model_name}.zip",
    f"/content/drive/MyDrive/RVC_Backup/{model_name}.zip",
)
%cd /content
shutil.rmtree("/content/zips")

# Resume-training

In [ ]:
# @title Load a Backup
from google.colab import drive
import os
import shutil

# @markdown Put the exact name you put as your Model Name in Applio.
modelname = "My-Project"  # @param {type:"string"}
source_path = "/content/drive/MyDrive/RVC_Backup/" + modelname
destination_path = "/content/Applio/logs/" + modelname
backup_timestamps_file = "last_backup_timestamps.txt"
if not os.path.exists(source_path):
    print(
        "The model folder does not exist. Please verify the name is correct or check your Google Drive."
    )
else:
    time_ = os.path.join("/content/drive/MyDrive/RVC_Backup/", backup_timestamps_file)
    time__ = os.path.join("/content/Applio/logs/", backup_timestamps_file)
    if os.path.exists(time_):
        shutil.copy(time_, time__)
    shutil.copytree(source_path, destination_path)
    print("Model backup loaded successfully.")

In [ ]:
# @title Set training variables
# @markdown ### ➡️ Use the same as you did previously
model_name = "Darwin"  # @param {type:"string"}
sample_rate = "40k"  # @param ["32k", "40k", "48k"] {allow-input: false}
rvc_version = "v2"  # @param ["v2", "v1"] {allow-input: false}
f0method = "rmvpe"  # @param ["pm", "dio", "crepe", "crepe-tiny", "harvest", "rmvpe"] {allow-input: false}
hop_length = 128  # @param {type:"slider", min:1, max:512, step:0}
sr = int(sample_rate.rstrip("k")) * 1000